In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import json
import cv2
from tqdm import tqdm
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
from results import *

expected_num_trials = 50
expected_num_participants = 59
participant_FPS = 30
db_path = "/Users/arijitdasgupta/Desktop/projects/red_green_completed_human_data/pilot_final_v1_redgreen.db"  # Update with the path to your .db file
demographic_path = "/Users/arijitdasgupta/Desktop/projects/red_green_completed_human_data/pilot_final_v1_demographic.csv"
path_to_data = '/Users/arijitdasgupta/Desktop/projects/mental_physics_trials/pilot_final'
session_df, trial_df, keystate_df, rgplot_df, valid_trial_ids, global_trial_names = extract_human_data(db_path, path_to_data)
# occlusion_durations, occlusion_frames, continuous_occlusion_periods, all_periods_seconds = extract_occlusion_data(path_to_data, participant_FPS)

In [2]:
path_to_data = '/Users/arijitdasgupta/Desktop/projects/mental_physics_trials/pilot_final'
import os
import numpy as np
entries = os.listdir(path_to_data)
e_folders = sorted([entry for entry in entries if entry.startswith('E')])
e_paths = [os.path.join(os.path.join(path_to_data, entry), 'data.npz') for entry in e_folders]
data_npzs = [np.load(e_path, allow_pickle=True) for e_path in e_paths]
position_data = {}
for i in range(len(e_paths)):
    position_data[e_folders[i]] = {
        int(k): {'x': v['x'], 'y': v['y']} for 
        k, v in data_npzs[i].get("step_data", {}).item().items()}

In [ ]:
path_to_data = '/Users/arijitdasgupta/Desktop/projects/mental_physics_trials/pilot_final'

entries = os.listdir(path_to_data)
e_folders = sorted([entry for entry in entries if entry.startswith('E')])
e_paths = [os.path.join(os.path.join(path_to_data, entry), 'data.npz') for entry in e_folders]
data_npzs = [np.load(e_path, allow_pickle=True) for e_path in e_paths]

reds = [x.get("rg_outcome").item() == 'red' for x in data_npzs]
greens = [x.get("rg_outcome").item() == 'green' for x in data_npzs]

sum(reds), sum(greens)

In [5]:
rgplot_df.to_csv('pilot_final_v1_rg_data.csv')

In [ ]:
#####################################################
# Plot pairwise RG plot animations with the stimuli #
#####################################################

# NOTE: defunct

# pairs = sorted(list(set([x[:-1] for x in global_trial_names])))
# for pair in pairs[22:]:
# # for pair in selected_pairs:
#     trial_a = f"{pair}a"
#     trial_b = f"{pair}b"
#     print(f"Displaying Pair: {trial_a} and {trial_b}")
#     trial_a_data = rgplot_df[rgplot_df["global_trial_name"] == trial_a]
#     trial_b_data = rgplot_df[rgplot_df["global_trial_name"] == trial_b]
#     animation_html = plot_pair_with_video(path_to_data, trial_a, trial_a_data, trial_b, trial_b_data,participant_FPS)
#     display(animation_html)

In [ ]:
###########################################################################################
# Unit test to ensure no duplicated trials and all trials have the appropriate data count #
##########################################################################################
duplicates = find_duplicate_completed_trials(trial_df)
if not duplicates.empty:
    print("Duplicate completed trials found:")
    print(duplicates)
else:
    print("No duplicate completed trials found.")
trial_counts = count_completed_trials_by_global_name(trial_df)
trial_counts[trial_counts['count'] != expected_num_participants//2]

In [ ]:
#######################################################################
# Distribution of scores identified in CURRENT valid participant data #
#######################################################################
# plot_scores_distribution(trial_df)

In [ ]:
####################
# Demographic Data #
####################
print_demo_data(session_df, demographic_path)

In [17]:
demo_df = pd.read_csv(demographic_path)
valid_demo_df = session_df.merge(demo_df, left_on='prolific_pid', right_on='Participant id')

In [ ]:
ages = np.array(valid_demo_df['Age'], dtype=int)
average_scores = np.array(valid_demo_df['average_score'], dtype=float)
sorted_indices = np.argsort(ages)
ages = ages[sorted_indices]
average_scores = average_scores[sorted_indices]
# Create a full range of ages (fill in the missing ones)
full_ages = np.arange(ages.min(), ages.max() + 1)

# Map the scores to the corresponding full ages (using NaN for missing values)
full_scores = np.full_like(full_ages, np.nan)  # Initialize with NaN

# Map the available scores to the full_ages
for i, age in enumerate(ages):
    index = np.where(full_ages == age)[0][0]
    full_scores[index] = average_scores[i]

# Create an evenly spaced x-axis (this will not be dependent on the actual ages)
evenly_spaced_x = np.arange(len(full_ages))
plt.figure(figsize=(18, 6))

# Plotting
plt.scatter(evenly_spaced_x, full_scores, label='Average Scores')

# Add labels and title
plt.xlabel('Age (Evenly Spaced)')
plt.ylabel('Average Score')
plt.ylim(20, 100)
plt.title('Scatter Plot with Missing Ages Filled In')
plt.xticks(evenly_spaced_x, full_ages)  # Use full_ages as x-ticks
# Show plot
plt.show()

In [ ]:
[trial_name for trial_name in occlusion_durations.keys() if len(continuous_occlusion_periods[trial_name]) == 1]

In [3]:
import pickle

# Save to a pickle file
with open("pilot_final_v1_human_data.pkl", "wb") as f:
    pickle.dump({"Session": session_df, "Trial": trial_df, "KeyState": keystate_df,
        'occlusion_durations' : occlusion_durations, 'continuous_occlusion_periods':
        continuous_occlusion_periods, 'occlusion_frames': occlusion_frames,
        'all_periods_seconds': all_periods_seconds, 'position_data': position_data}, f)


In [8]:
import cv2
import os
from tqdm import tqdm

def convert_video_fps(input_path, output_path=None, target_fps=30.0):
    """
    Convert a video to a target FPS using OpenCV.
    
    Args:
        input_path (str): Path to input video file
        output_path (str, optional): Path for output video. If None, will append '_30fps' to input filename
        target_fps (float, optional): Target frames per second. Defaults to 30.0
    
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Generate output path if not provided
        if output_path is None:
            filename, ext = os.path.splitext(input_path)
            output_path = f"{filename}_30fps{ext}"
            
        # Open the input video
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"Error: Could not open video file {input_path}")
            return False
            
        # Get video properties
        original_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        print(f"Original FPS: {original_fps}")
        print(f"Target FPS: {target_fps}")
        
        # Create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, target_fps, (width, height))
        
        # Process the video
        with tqdm(total=frame_count, desc="Converting video") as pbar:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                    
                # Write the frame
                out.write(frame)
                pbar.update(1)
        
        # Release everything
        cap.release()
        out.release()
        
        print(f"Conversion complete! Output saved to: {output_path}")
        return True
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False
    

def batch_convert_videos(video_paths, target_fps=30.0):
    """
    Convert multiple videos to target FPS.
    
    Args:
        video_paths (list): List of paths to video files
        target_fps (float, optional): Target frames per second. Defaults to 30.0
    """
    for video_path in video_paths:
        print(f"\nProcessing: {video_path}")
        success = convert_video_fps(video_path, target_fps=target_fps)
        if not success:
            print(f"Failed to convert: {video_path}")

In [9]:
vid_paths = []
for i in range(1,51):
    tname = 'E' + str(i)
    vid_paths.append(f'/Users/arijitdasgupta/Desktop/projects/mental_physics_trials/pilot_final/{tname}/high_res_obs.mp4')